## In this lab session we will implement two algorithms (an algorithm for sorting and one for detecting communities) each in two different ways: a naive, less efficient way and a better, faster way

### Fill in the CODE_HERE placeholders and check your code by evaluating the boxes with assert statements

### Make sure to evaluate every box

In [ ]:
#let us first install the necessary libraries
!pip install matplotlib
!pip install plotly
!pip install networkx

In [ ]:
#let us import the necessary libraries
from collections import defaultdict
from itertools import count

import time
import copy
import numpy as np

import matplotlib.pyplot as plt
import plotly.offline as py
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import networkx as nx

### Let's start with the sorting algorithm
We will create multiple data sets of integers we want to sort <br>
Each data set bigger than the previous one <br>
These different sized data sets will be used to see how computation time increases with the increasing size of the data set

In [ ]:
def apply_and_time_sorting(datasets, sort_function):
    datasets = copy.deepcopy(datasets)
    timings = []
    for dataset in datasets:
        start = time.time()
        sort_function(dataset)
        end = time.time()
        time_difference = end - start
        timings.append(time_difference)
    return timings

data_sizes = [5000,10000,20000]

In [ ]:
datasets = []
for data_size in data_sizes:
    datasets.append(np.random.permutation(data_size))

### Here we will implement the naive insertion sorting algorithm 
It's an algorithm that sorts an array in-place by iterating over the elements and inserting the current element
in the part of the array that is already sorted

```python
[1,11,13,7,4,8,0]
```

If the current element is 7, then the part of the algorithm before 7 is already sorted by the algorithm <br>
It then tries to find the position to insert the element 7 in the part of the array that is already sorted <br>
Resulting in

```python
[1,7,11,13,4,8,0]
```

The next element to be sorted is 4

In [ ]:
# define naive sort
def naivesort(data):
    """
    sorts the data in place
    """
    for i, element in enumerate(data):
        j = i
        #iterate back into the array as long as the current element is smaller than the previous one
        while j>0 and CODE_HERE: 
            data[j] = CODE_HERE #put the previous larger element in the position of the current one
            j -= 1
        data[j] = CODE_HERE # store the current element in its sorted position

In [ ]:
# perform naivesort on data
data = np.random.permutation(10)
naivesort(data)
assert np.all(data == np.array([0,1,2,3,4,5,6,7,8,9]))

In [ ]:
# perform naivesort on datasets with different sizes and store timings
timings_slow = apply_and_time_sorting(datasets, naivesort)
print(timings_slow)

### Let's compare this with the quicksort algorithm that was discussed in class

In [ ]:
datasets = []
for data_size in data_sizes:
    datasets.append(np.random.permutation(data_size))

In [ ]:
# define functions
def swap(l, i, j):
    """
    swaps element in position i to position j and vice versa in the list l
    """
    tmp = l[j]
    l[j] = l[i]
    l[i] = tmp

def quicksort_helper(l, starti, endi):
    #set_trace()
    if starti >= endi:
        return
    #if the data set contains less than 100 elements, using the naive insertion sort is better
    if endi-starti+1<=100:
        naivesort(l)
    else:
        middlei = int(0.5*(starti+endi))
        #first make sure the start, middle and end element are in sorted order
        if (l[middlei]<l[starti]):
            swap(l,CODE_HERE, CODE_HERE)
        if (l[endi]<l[starti]):
            swap(l, CODE_HERE, CODE_HERE)
        if (l[endi]<l[middlei]):
            swap(l, CODE_HERE, CODE_HERE)
        pivoti = middlei
        
        #put the pivot element just before the last 
        swap(l, CODE_HERE, CODE_HERE)
        pivot = l[endi-1]
        lefti = starti+1
        righti = endi-2
        
        #now make sure the elements are partially sorted,
        #i.e. elements in the first half should be smaller than the pivot element
        #the elements in the second half should be larger than the pivot element
        while True:
            #move the lefti cursor from left to right until you encounter an element larger than the pivot
            while l[lefti]<pivot: 
                CODE_HERE
            #move the righti cursor from right to left until you encounter an element smaller than the pivot
            while pivot<l[righti]: 
                CODE_HERE
            #if the lefti cursor is still lower than the righi cursor, swap them, otherwise break out of the loop
            if lefti < righti:  
                swap(l, CODE_HERE, CODE_HERE)
            else:
                break
                
        pivoti = lefti
        swap(l, pivoti, endi-1)
        #recursive sort the first half of the array
        quicksort_helper(l, starti, pivoti-1)
        #recursive sort the second half of the array
        quicksort_helper(l, pivoti+1, endi)
        
def quicksort(l):
    quicksort_helper(l, 0, len(l)-1)

In [ ]:
# perform quicksort on data
data = np.random.permutation(10)
quicksort(data)
assert np.all(data == np.array([0,1,2,3,4,5,6,7,8,9]))

In [ ]:
# perform quicksort on datasets with different sizes and store timings
timings_fast = apply_and_time_sorting(datasets, quicksort)
print(timings_fast)

#### Now plot the timings for the naive and quicksort algorithm. What does this tell you about the complexity of the algorithms?

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_sizes,
                y=timings_slow,
                mode='lines',
                name='slow'))
fig.add_trace(go.Scatter(x=data_sizes,
                y=timings_fast,
                mode='lines',
                name='fast'))
fig.show()

## The next algorithm we will investigate is the community detection algorithm: the Louvain method

### As a simple example we will take the social network of a karate club collected by Zachary, the members have a link between them in case they interacted with each other outside the club. The goal is to find the communities that exist in the club

First download the file from https://raw.githubusercontent.com/ddvlamin/data_engineering_ua/master/code_examples/week4/data/karate_edges_78.txt
and put it in the same folder as the notebook

In [ ]:
file = "./karate_edges_78.txt"

### Let's first create the adjacency matrix for the social network

In [ ]:
def create_adjacency_matrix(file):
    node_neighbors = defaultdict(list)

    with open(file, "r") as fin:
        for line in fin:
            node_1, node_2 = line.strip().split('\t')
            node_1 = int(node_1)-1 #offset to make the integers start from zero
            node_2 = int(node_2)-1
            node_neighbors[node_1].append(node_2)

    number_of_nodes = len(node_neighbors)
    A = np.zeros((number_of_nodes, number_of_nodes))

    for node, neighbors in node_neighbors.items():
        for neighbor in neighbors:
            A[node, neighbor] = 1

    return A

In [ ]:
A = create_adjacency_matrix(file)

### Compute the total sum of edges in the adjacency matrix A (divided by 2)
Hint: use np.sum

In [ ]:
m = CODE_HERE

### Let's now define a function to compute the modularity of the network given the adjacency matrix A, the communities and the total sum of edge weights
Hint: look at the modularity formula in the course slides

In [ ]:
def compute_modularity(A, communities, m):
    """
    computes the modularity for a given network A and communities communities (see formula in slides)
    
    A: dense square matrix, nonzero element on row i and column j indicating 
        a weighted connection between vertex i and j
    communities: vector of communities, on position i the community for node i, so the length of the list equals 
        the number of nodes in the graph
    m: an integer that equals the sum of all edges in the graph divided by two
    """
    nrows, ncols = A.shape
    assert nrows == ncols
    nvertices = nrows

    modularity = 0.0
    #iterate over all pair of vertices
    for vertexi in range(nvertices):
        for vertexj in range(nvertices):
            #sum all the edge weight of vertexi
            ki = CODE_HERE
            #sum all the edge weight of vertexj
            kj = CODE_HERE
            #if both vertices are in the same community add a term to the modularity
            if communities[vertexi] == communities[vertexj]:
                modularity += CODE_HERE

    return modularity/(2*m)

### Compute the modularity putting every vertex/node in its own community

The community vector has the same length as the number of vertices/nodes in the graph <br>
Position i in the community vector defines the community of vertex/node i <br>
e.g. [0,0,1,1,1] means the first two vertices belong the community 0 and the last three to community 1

In [ ]:
def initialize_communities(n_vertices):
    return CODE_HERE # use np.arange to create a vector from 0 to n_vertices-1
    
initial_communities = initialize_communities(A.shape[0])

### Now compute the modularity for the given adjancency matrix, the initial communities and the total sum of edges

In [ ]:
initial_modularity = compute_modularity(A, initial_communities, m)

In [ ]:
assert initial_modularity == -0.04980276134122286

### Now we will implement the first phase of the Louvain method but using a naive, inefficient way of computing the modularity difference

Let's first define some helper function to perform the local search (i.e. the first phase) <br>

The first helper function let's us compute the difference in modularity when we move a node from its current
community into another community <br>

We will (on purpose) compute this in a very naive inefficient way

In [ ]:
def modularity_difference(A, communities, m, modularity, vertex, community):
    """
    A: the adjacency matrix of the graph
    communities: vector of communities, on position i the community for node i, so the length of the list equals 
        the number of nodes in the graph 
    m: an integer that equals the sum of all edges in the graph divided by two
    modularity: the modularity of the graph A with communities defined by the variable communities
    vertex: the index of the vertex/node you want to compute the modularity difference for when moving it to community
    community: the community to which you want move the vertex
    """
    #store the vertex' community in a variable
    original_community = CODE_HERE 
    #change community of the current vertex
    communities[vertex] = CODE_HERE
    #compute the new modularity use the compute_modularity function
    new_modularity = CODE_HERE
    #compute the difference between the new modulariy and the old one
    modularity_difference = CODE_HERE
    #restore original community of the vertex
    communities[vertex] = CODE_HERE
    return modularity_difference

In [ ]:
test_A = np.array([
    [0,1,1,0,0,0],
    [1,0,1,0,1,0],
    [1,1,0,0,0,0],
    [0,0,0,0,1,1],
    [0,1,0,1,0,1],
    [0,0,0,1,1,0]
    ]
)
test_communities = np.arange(0,6)
test_m = 0.5*np.sum(A)
test_mod = compute_modularity(test_A, test_communities, test_m)
expected = [0.0, 0.01232741617357002, 0.012491781722550954, -0.0003287310979618672, -0.0004930966469428009, -0.0003287310979618672]
for vertexi in range(0,6):
    assert expected[vertexi] == modularity_difference(test_A, test_communities, test_m, test_mod, 0, vertexi)

### Now use the modularity_difference function to iterate over all communities and see which community for the given vertex gives the biggest increase in modularity

In [ ]:
def find_best_community(A, communities, m, modularity, vertex):
    """
    A: the adjacency matrix of the graph
    communities: vector of communities, on position i the community for node i, so the length of the list equals 
        the number of nodes in the graph 
    m: an integer that equals the sum of all edges in the graph divided by two
    modularity: the modularity of the graph A with communities defined by the variable communities
    vertex: the index of the vertex/node for which you want to find the best community
    """    
    unique_communities = np.unique(communities)
    max_modularity_difference = 0
    best_community = communities[vertex]
    for community in unique_communities:
        #compute the modularity difference for the given vertex and the community by applying the 
        #before defined modularity_difference function
        modularity_diff = CODE_HERE
        #if the difference is larger than the maximum modularity difference, store the new difference
        #and set the best_community variable to this new community
        if modularity_diff > max_modularity_difference:
            max_modularity_difference = CODE_HERE
            best_community = CODE_HERE
    return best_community, max_modularity_difference

In [ ]:
test_A = np.array([
    [0,1,1,0,0,0],
    [1,0,1,0,1,0],
    [1,1,0,0,0,0],
    [0,0,0,0,1,1],
    [0,1,0,1,0,1],
    [0,0,0,1,1,0]
    ]
)
test_communities = np.arange(0,6)
test_m = 0.5*np.sum(A)
test_mod = compute_modularity(test_A, test_communities, test_m)
assert find_best_community(test_A, test_communities, test_m, test_mod, 0)[0] == 2
assert find_best_community(test_A, test_communities, test_m, test_mod, 4)[0] == 3

### We will now use the find_best_community function to keep iterating over all nodes until we can't increase the modularity anymore

In [ ]:
def reset_communities(communities):
    """
    reset the community vector so that the communities  start from zero
    
    Example:
    --------
    communities = [17,17,3,17,3,1,1,1,3,17]
    reset_communities(communities) == [0,0,1,0,1,2,2,2,1,0]
    """
    counter = count()
    community_mapper = defaultdict(lambda: next(counter))
    reset_communities = np.zeros((len(communities),), dtype=int)
    for vertex, community in enumerate(communities):
        reset_communities[vertex] = community_mapper[community]
    return reset_communities

def local_search(A, communities, m, modularity, seed=11):
    #we iterate over the vertices in a randomized way
    nvertices = len(communities)
    if seed>0:
        vertex_order = np.random.RandomState(seed=seed).permutation(nvertices)
    else:
        vertex_order = range(nvertices)

    is_modularity_increasing = True
    #keep iterating over the vertices until the modulariy stops increasing
    while is_modularity_increasing:
        is_modularity_increasing = False
        for vertex in vertex_order:
            #find the best community for the vertex using the find_best_community function
            best_community, modularity_difference = CODE_HERE
            #if the modulariy is strictly positive set the community of the vertex to the best_community
            if modularity_difference > 0:
                communities[vertex] = CODE_HERE
                modularity += CODE_HERE #update the modularity with the difference
                is_modularity_increasing = True #set the flag that indicates the modularity is still increasing

    communities = reset_communities(communities) #just reindexing the new communities so they start from zero

    return communities, modularity

In [ ]:
test_A = np.array([
    [0,1,1,0,0,0],
    [1,0,1,0,1,0],
    [1,1,0,0,0,0],
    [0,0,0,0,1,1],
    [0,1,0,1,0,1],
    [0,0,0,1,1,0]
    ]
)
test_communities = np.arange(0,6)
test_m = 0.5*np.sum(A)
test_mod = compute_modularity(test_A, test_communities, test_m)
assert np.all(local_search(test_A, test_communities, test_m, test_mod)[0] == np.array([0,0,0,1,1,1]))

### Next we implement the second phase of the Louvain method
This step reduces the network where all nodes, that belong to the same community, are now reduced into one node <br>
The new weight between two nodes is the sum of all the edge weights between their respective communities <br>
The weight of the self-link is the sum of all weights between the original nodes of the community <br>

Example <br>

Original adjacency matrix <br>
```python
    [
        [0,1,1,0],
        [1,0,1,0],
        [1,1,0,1],
        [0,0,1,0]
    ]
```
with communities <br>
```python
    [0,0,0,1]
```
will reduce to <br>
```python
    [
        [6,1],
        [1,0]
    ]
```
with communities <br>
```python
    [0,1]
```

In [ ]:
def reduce_network(A, communities):
    #assumes communities starting from zero increasing by one
    unique_communities = np.unique(communities)
    n_communities = len(unique_communities)
    #initialize the new reduced adjacency matrix as a zero square matrix of dimension n_communities
    reduced_A = CODE_HERE

    #now let's fill in all the entries in the reduced_A matrix 
    #(note that this can be done more efficiently because the matrix is symmetric) 
    for communityi in unique_communities:
        for communityj in unique_communities:
            #find all the vertex indices that belong to communityi 
            #(hint: use np.argwhere and don't forget to flatten the array)
            vertices_i = CODE_HERE
            #find all the vertex indices that belong to communityj 
            vertices_j = CODE_HERE
            #select the sub matrix of all the vertices in communityi and communityj
            sub_A = A[vertices_i,:]
            sub_A = sub_A[:,vertices_j]
            #the edge weight between communityi and communityj is the sum of the weights
            #between all the vertices of communities communityi and communityj
            reduced_A[communityi, communityj] = CODE_HERE

    #put each vertex in the reduced network again in its own community
    communities = np.array([i for i in range(reduced_A.shape[0])], dtype=int)
    
    return reduced_A, communities

In [ ]:
test_A = np.array([
    [0,1,1,0,0,0],
    [1,0,1,0,1,0],
    [1,1,0,0,0,0],
    [0,0,0,0,1,1],
    [0,1,0,1,0,1],
    [0,0,0,1,1,0]
    ]
)
test_communities = [0,0,0,1,1,1]
assert np.all(reduce_network(test_A, test_communities)[0] == np.array([[6,1],[1,6]]))

### Now we will combine the local_search function and the reduce_function to find the community structure with the highest modularity

In [ ]:
def traceback_communities(reduced_communities, communities):
    #assumes community i after local search is also reduced to the i-th node
    new_communities = np.zeros((len(communities,)),dtype=int)
    for vertex, community in enumerate(communities):
        new_communities[vertex] = reduced_communities[community]
    return new_communities

def detect_communities(A, communities):
    m = 0.5*np.sum(np.sum(A))
    modularity = compute_modularity(A, communities, m)

    #first do a local_search to find better community structure using the local_search function
    local_communities, modularity = CODE_HERE

    #as long as we are able to further combine communities, we keep optimizing
    if len(np.unique(local_communities)) < len(communities):
        #reduce the network with the communities found by the previous local search
        reduced_A, reduced_communities = CODE_HERE
        #keep repeating these two phases (local_search and reduce_network) 
        #by a recursive call to detect_communities using the reduced network reduced_A and
        #the reduced communities reduced_communities
        #if this recursion makes your head explode, that's normal, don't worry
        reduced_communities = CODE_HERE
        #a function to trace back the communities of the original nodes using the reduced network communities
        expanded_communities = traceback_communities(reduced_communities, local_communities)
        return expanded_communities
    else:
        return communities

### Let's now apply the algorithm to the Karate Club network and compute the new modularity
Let's also time it to see how slow it is

In [ ]:
initial_communities = initialize_communities(A.shape[0])

start_time = time.time()
best_communities = detect_communities(A, initial_communities)
end_time = time.time()
time_slowest = end_time-start_time
print(f"Took {time_slowest} seconds to execute")

In [ ]:
assert compute_modularity(A, best_communities, m) > 0.4

### Let's visualize the network with the optimal commmunities using the below functions

In [ ]:
def create_graph(A, communities):
    G=nx.Graph()
    
    for nodei in range(A.shape[0]):
        d = {"community": communities[nodei]}
        G.add_node(nodei+1, **d)
    
    for i, j in zip(*A.nonzero()):
        if communities[i] == communities[j]:
            weight = 100
        else:
            weight = 1
        G.add_edge(i+1, j+1, weight=weight)
        
    return G

def plot_graph(G, pos=None):
    plt.figure(figsize=(30,15))  
    
    node_communities = [d["community"] for _, d in G.nodes(data=True)]
    
    if pos==None:
        pos = nx.spring_layout(G, scale=30)  # positions for all nodes
    
    nx.draw_networkx_nodes(G, pos, node_color=node_communities)
    nx.draw_networkx_edges(G, pos)

    nx.draw_networkx_labels(G, pos, font_size=30, font_family='sans-serif')

    plt.axis('off')
    plt.show()
    
    return pos

G = create_graph(A, best_communities)
plot_graph(G)

### Let's now replace the naive computation of the modularity difference with the difference formula presented in the paper

Link to paper: https://www.researchgate.net/publication/1913681_Fast_Unfolding_of_Communities_in_Large_Networks

In [ ]:
def modularity_difference(A, communities, m, modularity, vertex, community):
    """
    A: the adjacency matrix of the graph
    communities: vector of communities, on position i the community for node i, so the length of the list equals 
        the number of nodes in the graph 
    m: an integer that equals the sum of all edges in the graph divided by two
    modularity: the modularity of the graph A with communities defined by the variable communities
    vertex: the index of the vertex/node you want to compute the modularity difference for when moving it to community
    community: the community to which you want move the vertex
    """
    #store the vertex' community in a variable
    old_community = communities[vertex] 
    
    #True for all the nodes that belong to the new community to which we want to move our current vertex
    new_community_nodes = communities == community
    new_community_nodes[vertex] = False
    #True for all the nodes that belong to the old community of the vertex
    old_community_nodes = communities == old_community
    old_community_nodes[vertex] = False
    
    new_k_i_in = CODE_HERE #sum all the weights of edges going from the vertex to nodes of its new community
    old_k_i_in = CODE_HERE #sum all the weights of edges going from the vertex to nodes of its old community
    
    ki = np.sum(A[vertex,:])
    
    new_sum_tot = CODE_HERE #sum all weights of edges incident to nodes of the new community
    old_sum_tot = CODE_HERE #sum all weights of edges incident to nodes of the old community
    
    #formula for the fast computation of the modularity difference
    modularity_difference = (1/m)*(new_k_i_in-old_k_i_in) + (1/(2*m*m))*ki*(old_sum_tot-new_sum_tot)
    
    return modularity_difference

In [ ]:
test_A = np.array([
    [0,1,1,0,0,0],
    [1,0,1,0,1,0],
    [1,1,0,0,0,0],
    [0,0,0,0,1,1],
    [0,1,0,1,0,1],
    [0,0,0,1,1,0]
    ]
)
test_communities = np.arange(0,6)
test_m = 0.5*np.sum(A)
test_mod = compute_modularity(test_A, test_communities, test_m)
expected = [
    (0.0,0.0), 
    (0.012327416173,0.012327416174), 
    (0.012491781722,0.012491781723), 
    (-0.00032873110,-0.00032873109), 
    (-0.00049309665,-0.00049309664), 
    (-0.00032873110,-0.00032873109)
]
for vertexi in range(0,6):
    mod_diff = modularity_difference(test_A, test_communities, test_m, test_mod, 0, vertexi)
    lb, ub = expected[vertexi]
    assert lb <= mod_diff <= ub

In [ ]:
initial_communities = initialize_communities(A.shape[0])
start_time = time.time()
best_communities = detect_communities(A, initial_communities)
end_time = time.time()
time_faster = end_time-start_time
print(f"Took {time_faster} seconds to execute")

In [ ]:
assert compute_modularity(A, best_communities, m) > 0.4

### We will now compare this naive implementation with a fast implementation of the Louvain method:
https://python-louvain.readthedocs.io/en/latest/

In [ ]:
# install the library
!pip install python-louvain

In [ ]:
# import library
import community

In [ ]:
# convert adjacency matrix to graph
G = nx.from_numpy_matrix(A)

#### Read the documentation to find out which command to use to compute the partition using the fast louvain library. Compare execution time to the naive implementation

In [ ]:
# compute the best partition
start_time = time.time()
partition = CODE_HERE
time_fastest = time.time()-start_time
print(f"Took {time_fastest} seconds to execute")

#### Visualize the network using the previously defined functions and compare this to the previous solution. What do you see?

In [ ]:
# visualize network
CODE_HERE

#### Visualize the computation times in a bar chart

In [ ]:
fig = go.Figure(data=[go.Bar(
            x=["slow","faster","fastest"], y=[time_slowest, time_faster, time_fastest],
            text=[time_slowest, time_faster, time_fastest],
            textposition='auto',
        )])

fig.show()